## Learning to fine-tune Mistral

In [2]:
!pip install mistralai pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 579.0 kB/s eta 0:00:00 kB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 1.7 MB/s eta 0:00:001.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 1.9 MB/s eta 0:00:000:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 2.8 MB/s eta 0:00:002.9 MB/s eta 0:00:01
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.12
    Uninstalling pydantic-1.10.12:
      Successfully uninstalled pydantic-1.10.12


## Prepare the dataset 

In [159]:
import pandas as pd
df = pd.read_parquet('https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k/resolve/main/data/test_gen-00000-of-00001-3d4cd8309148a71f.parquet')

df = df.sample(1000)

df_train=df.sample(frac=0.995,random_state=200)
df_eval=df.drop(df_train.index)

df_train.to_json("ultrachat_chunk_train.jsonl", orient="records", lines=True)
df_eval.to_json("ultrachat_chunk_eval.jsonl", orient="records", lines=True)

In [94]:
df_train

,prompt,prompt_id,messages
2676,Create a comprehensive beginner's guide to cha...,f626f2d0dbd790669ef1cdc31b0dcb813d34d79bd2d54d...,[{'content': 'Create a comprehensive beginner'...
27670,The relationship between Singaporean and their...,fd117ce91a3d1b48173d48499bd77c55792451f59c778f...,[{'content': 'The relationship between Singapo...
12322,Provide step-by-step instructions on setting u...,0967aa2a1974e0e05c2c7f7e388fa9d749533e92d5d15f...,[{'content': 'Provide step-by-step instruction...
16464,Can you provide a summary of the essay writing...,f2effaec1d4c9d41b0ab29daf37299f1dc76da4eee1b91...,[{'content': 'Can you provide a summary of the...
2298,How has Porsche's customer service contributed...,6cea29c689c378a7d6fa551ebc24c83ff0c2b7969d41fb...,[{'content': 'How has Porsche's customer servi...
...,...,...,...
3917,Could you paraphrase the key points of Arpawon...,001edb8f538da64ceb76f92d32a9be05e69e64297914c5...,[{'content': 'Could you paraphrase the key poi...
22822,Generate response to the question/instruction ...,a26c56c75e9bc1eaa71125d25e23edd207ce9a548808df...,[{'content': 'Generate response to the questio...
7405,Create a tabbed content section on the homepag...,88aa0bdd643e79cd25219cdae6861579da96e44dd6c30c...,[{'content': 'Create a tabbed content section ...
18301,"Write a fictional short story of at least 1,50...",bc0b4b6b6907c5a71ae23cc811424de52db458ff39f9fd...,[{'content': 'Write a fictional short story of...


In [84]:

df = pd.read_json("data.json")

In [88]:
flattened_df = df.stack().reset_index(drop=True).to_frame(name='Values')

In [113]:
flattened_df.sample(2)

,Values
1372,{'question': 'How do fixed-rate mortgages vary...
814,{'question': 'How can I benefit from China's d...


In [131]:

# Convert the DataFrame
def transform_data(df):
    transformed = []
    for index, row in df.iterrows():
        try:
            qa_pair = row['Values']  # Accessing the first element of the list in the cell
            transformed.append({'content': qa_pair['question'], 'role': 'user'})
            transformed.append({'content': qa_pair['answer'], 'role': 'assistant'})
        except:
            pass
    return pd.DataFrame(transformed)

# Call the function and create the new DataFrame
new_df = transform_data(flattened_df)
print(new_df)

                                                content       role
0     What does it mean when the economy is characte...       user
1     Steady but slow growth indicates a stable yet ...  assistant
2     How do changes in inflation affect our financi...       user
3     Disinflation (slowdown in inflation) can lead ...  assistant
4     What are some key factors that can impact the ...       user
...                                                 ...        ...
8999  Regulatory authorities can ensure resilience b...  assistant
9000        Why is it important to implement Basel III?       user
9001  Implementing Basel III is crucial because it s...  assistant
9002  What kind of structural reforms are needed to ...       user
9003  To boost medium-term growth, economies need ta...  assistant

[9004 rows x 2 columns]


In [152]:
df = new_df

df = df.sample(1000)

df_train=df.sample(frac=0.995,random_state=200)
df_eval=df.drop(df_train.index)

df_train.to_json("ultrachat_chunk_train.jsonl", orient="records", lines=True)
df_eval.to_json("ultrachat_chunk_eval.jsonl", orient="records", lines=True)

In [141]:
#!ls -lh

total 1.8M
-rw-rw-r-- 1 giskard giskard 1.5M Jun 24 14:29 data.json
-rw-rw-r-- 1 giskard giskard  54K Jun 26 22:03 FinetuningV1.ipynb
-rw-rw-r-- 1 giskard giskard 3.4K Jun 26 21:39 reformat_data.py
-rw-rw-r-- 1 giskard giskard 3.4K Jun 26 21:43 reformat_data.py.1
-rw-rw-r-- 1 giskard giskard 3.4K Jun 26 21:43 reformat_data.py.2
-rw-rw-r-- 1 giskard giskard  450 Jun 26 22:03 ultrachat_chunk_eval.jsonl
-rw-rw-r-- 1 giskard giskard 160K Jun 26 22:03 ultrachat_chunk_train.jsonl
-rw-rw-r-- 1 giskard giskard  35K Jun 24 14:29 Untitled.ipynb


In [143]:
# download the validation and reformat script
#!wget https://raw.githubusercontent.com/mistralai/mistral-finetune/main/utils/reformat_data.py

--2024-06-26 22:03:36--  https://raw.githubusercontent.com/mistralai/mistral-finetune/main/utils/reformat_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8002::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3381 (3.3K) [text/plain]
Saving to: ‘reformat_data.py.3’

reformat_data.py.3  100%[===================>]   3.30K  --.-KB/s    in 0s      

2024-06-26 22:03:36 (8.13 MB/s) - ‘reformat_data.py.3’ saved [3381/3381]



In [145]:
# validate and reformat the training data
#!python reformat_data.py ultrachat_chunk_train.jsonl

In [147]:
# validate the reformat the eval data
#!python reformat_data.py ultrachat_chunk_eval.jsonl

In [149]:
#df_train.sample(1)['messages'].values

KeyError: 'messages'

## Upload dataset

In [157]:
import os
from mistralai.client import MistralClient

api_key = "xJpeeVFY3eoNskGHu1x6rj3Cwu9uVEhV"
client = MistralClient(api_key=api_key)

with open("ultrachat_chunk_train.jsonl", "rb") as f:
    ultrachat_chunk_train = client.files.create(file=("ultrachat_chunk_train.jsonl", f))
with open("ultrachat_chunk_eval.jsonl", "rb") as f:
    ultrachat_chunk_eval = client.files.create(file=("ultrachat_chunk_eval.jsonl", f))

MistralAPIException: Status: 422. Message: {"detail": "Invalid file format.", "description": "Found at least 994 errors in this file. You can view supported formats here: https://docs.mistral.ai/capabilities/finetuning. ", "errors": [{"message": "1 validation error for FinetuningMessages messages   Input should be a valid list [type=list_type, input_value={'content': 'What role do...casts?', 'role': 'user'}, input_type=dict]     For further information visit https://errors.pydantic.dev/2.6/v/list_type", "line_number": 1}, {"message": "1 validation error for FinetuningMessages messages   Input should be a valid list [type=list_type, input_value={'content': 'As trade vol....', 'role': 'assistant'}, input_type=dict]     For further information visit https://errors.pydantic.dev/2.6/v/list_type", "line_number": 2}, {"message": "1 validation error for FinetuningMessages messages   Input should be a valid list [type=list_type, input_value={'content': 'A positive c....', 'role': 'assistant'}, 

In [54]:
import json
def pprint(obj):
    print(json.dumps(obj.dict(), indent=4))

In [55]:
pprint(ultrachat_chunk_train)

{
    "id": "9620c41f-690c-428a-98cc-3e08030f5871",
    "object": "file",
    "bytes": 4209251,
    "created_at": 1719434651,
    "filename": "ultrachat_chunk_train.jsonl",
    "purpose": "fine-tune"
}


In [56]:
pprint(ultrachat_chunk_eval)

{
    "id": "bee47c87-5420-4497-8bd6-42145042df28",
    "object": "file",
    "bytes": 21051,
    "created_at": 1719434652,
    "filename": "ultrachat_chunk_eval.jsonl",
    "purpose": "fine-tune"
}


## Create a fine-tuning job

In [62]:
from mistralai.models.jobs import TrainingParameters

created_jobs = client.jobs.create(
    model="open-mistral-7b",
    training_files=[ultrachat_chunk_train.id],
    validation_files=[ultrachat_chunk_eval.id],
    hyperparameters=TrainingParameters(
        training_steps=10,
        learning_rate=0.0001,
        )
)

In [66]:
pprint(created_jobs)

{
    "id": "87632faf-a00a-45e8-bc83-d91ccbb70e80",
    "hyperparameters": {
        "training_steps": 10,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": null,
    "model": "open-mistral-7b",
    "status": "QUEUED",
    "job_type": "FT",
    "created_at": 1719434655,
    "modified_at": 1719434655,
    "training_files": [
        "9620c41f-690c-428a-98cc-3e08030f5871"
    ],
    "validation_files": [
        "bee47c87-5420-4497-8bd6-42145042df28"
    ],
    "object": "job",
    "integrations": []
}


In [68]:
import time

retrieved_job = client.jobs.retrieve(created_jobs.id)
while retrieved_job.status in ["RUNNING", "QUEUED"]:
    retrieved_job = client.jobs.retrieve(created_jobs.id)
    pprint(retrieved_job)
    print(f"Job is {retrieved_job.status}, waiting 10 seconds")
    time.sleep(10)

{
    "id": "87632faf-a00a-45e8-bc83-d91ccbb70e80",
    "hyperparameters": {
        "training_steps": 10,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": null,
    "model": "open-mistral-7b",
    "status": "RUNNING",
    "job_type": "FT",
    "created_at": 1719434655,
    "modified_at": 1719434657,
    "training_files": [
        "9620c41f-690c-428a-98cc-3e08030f5871"
    ],
    "validation_files": [
        "bee47c87-5420-4497-8bd6-42145042df28"
    ],
    "object": "job",
    "integrations": [],
    "events": [
        {
            "name": "status-updated",
            "data": {
                "status": "RUNNING"
            },
            "created_at": 1719434657
        },
        {
            "name": "status-updated",
            "data": {
                "status": "QUEUED"
            },
            "created_at": 1719434655
        }
    ],
    "checkpoints": [],
    "estimated_start_time": null
}
Job is RUNNING, waiting 10 seconds
{
    "id": "87632faf-a00a-45

In [69]:
# List jobs
jobs = client.jobs.list()
pprint(jobs)

{
    "data": [
        {
            "id": "87632faf-a00a-45e8-bc83-d91ccbb70e80",
            "hyperparameters": {
                "training_steps": 10,
                "learning_rate": 0.0001
            },
            "fine_tuned_model": "ft:open-mistral-7b:904f6bd1:20240626:87632faf",
            "model": "open-mistral-7b",
            "status": "SUCCESS",
            "job_type": "FT",
            "created_at": 1719434655,
            "modified_at": 1719434742,
            "training_files": [
                "9620c41f-690c-428a-98cc-3e08030f5871"
            ],
            "validation_files": [
                "bee47c87-5420-4497-8bd6-42145042df28"
            ],
            "object": "job",
            "integrations": []
        }
    ],
    "object": "list"
}


In [70]:
# Retrieve a jobs
retrieved_jobs = client.jobs.retrieve(created_jobs.id)
pprint(retrieved_jobs)


{
    "id": "87632faf-a00a-45e8-bc83-d91ccbb70e80",
    "hyperparameters": {
        "training_steps": 10,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": "ft:open-mistral-7b:904f6bd1:20240626:87632faf",
    "model": "open-mistral-7b",
    "status": "SUCCESS",
    "job_type": "FT",
    "created_at": 1719434655,
    "modified_at": 1719434742,
    "training_files": [
        "9620c41f-690c-428a-98cc-3e08030f5871"
    ],
    "validation_files": [
        "bee47c87-5420-4497-8bd6-42145042df28"
    ],
    "object": "job",
    "integrations": [],
    "events": [
        {
            "name": "status-updated",
            "data": {
                "status": "SUCCESS"
            },
            "created_at": 1719434742
        },
        {
            "name": "status-updated",
            "data": {
                "status": "RUNNING"
            },
            "created_at": 1719434657
        },
        {
            "name": "status-updated",
            "data": {
              

## Use a fine-tuned model

In [72]:
from mistralai.models.chat_completion import ChatMessage

chat_response = client.chat(
    model=retrieved_jobs.fine_tuned_model,
    messages=[ChatMessage(role='user', content='What is the best French cheese?')]
)

In [73]:
pprint(chat_response)

{
    "id": "1223ccaf4741452c939c633c126a215a",
    "object": "chat.completion",
    "created": 1719434755,
    "model": "ft:open-mistral-7b:904f6bd1:20240626:87632faf",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "The best French cheese is a matter of personal preference. Many people enjoy a creamy Brie or Camembert, while others prefer the strong flavor of Roquefort or Comt\u00e9. Ultimately, the best French cheese is the one that you enjoy the most.",
                "name": null,
                "tool_calls": null,
                "tool_call_id": null
            },
            "finish_reason": "stop"
        }
    ],
    "usage": {
        "prompt_tokens": 10,
        "total_tokens": 65,
        "completion_tokens": 55
    }
}
